In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from time import time

In [2]:
act_train_data = pd.read_csv("./act_train.csv", 
                             dtype={'people_id': np.str, 'activity_id': np.str, 'outcome': np.int8}, 
                             parse_dates=['date'])
act_test_data  = pd.read_csv("./act_test.csv", 
                             dtype={'people_id': np.str, 'activity_id': np.str}, parse_dates=['date'])
people_data    = pd.read_csv("./people.csv", 
                             dtype={'people_id': np.str, 'activity_id': np.str, 'char_38': np.int32}, 
                             parse_dates=['date'])

In [3]:
act_whole = pd.concat([act_train_data, act_test_data], ignore_index=True)
del act_train_data, act_test_data

act_whole = act_whole[['activity_id', 'people_id', 'outcome', 'date']]
act_whole[2197285:2197300]

,activity_id,people_id,outcome,date
2197285,act2_4581579,ppl_99994,1.0,2023-01-13
2197286,act2_4668076,ppl_99994,1.0,2023-06-16
2197287,act2_4743548,ppl_99994,1.0,2023-03-30
2197288,act2_536973,ppl_99994,1.0,2023-01-19
2197289,act2_688656,ppl_99994,1.0,2023-05-02
2197290,act2_715089,ppl_99994,1.0,2023-06-15
2197291,act1_249281,ppl_100004,NaN,2022-07-20
2197292,act2_230855,ppl_100004,NaN,2022-07-20
2197293,act1_240724,ppl_10001,NaN,2022-10-14
2197294,act1_83552,ppl_10001,NaN,2022-11-27


In [4]:
act_merged = act_whole.merge(people_data, on='people_id', how='left')
del act_whole, people_data
act_merged[:3]

,activity_id,people_id,outcome,date_x,char_1,group_1,char_2,date_y,char_3,char_4,...,char_29,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38
0,act2_1734928,ppl_100,0.0,2023-08-26,type 2,group 17304,type 2,2021-06-29,type 5,type 5,...,False,True,True,False,False,True,True,True,False,36
1,act2_2434093,ppl_100,0.0,2022-09-27,type 2,group 17304,type 2,2021-06-29,type 5,type 5,...,False,True,True,False,False,True,True,True,False,36
2,act2_3404049,ppl_100,0.0,2022-09-27,type 2,group 17304,type 2,2021-06-29,type 5,type 5,...,False,True,True,False,False,True,True,True,False,36


In [5]:
act_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2695978 entries, 0 to 2695977
Data columns (total 44 columns):
activity_id    object
people_id      object
outcome        float64
date_x         datetime64[ns]
char_1         object
group_1        object
char_2         object
date_y         datetime64[ns]
char_3         object
char_4         object
char_5         object
char_6         object
char_7         object
char_8         object
char_9         object
char_10        bool
char_11        bool
char_12        bool
char_13        bool
char_14        bool
char_15        bool
char_16        bool
char_17        bool
char_18        bool
char_19        bool
char_20        bool
char_21        bool
char_22        bool
char_23        bool
char_24        bool
char_25        bool
char_26        bool
char_27        bool
char_28        bool
char_29        bool
char_30        bool
char_31        bool
char_32        bool
char_33        bool
char_34        bool
char_35        bool
char_36        bool


In [6]:
%%time
allDays = pd.Series(pd.date_range(act_merged['date_x'].min(), act_merged['date_x'].max()))
nb_days = len(allDays)
# create all combinations of companies (group_1) and dates
allCompDays = [pd.concat([pd.Series([g]*nb_days), allDays], axis=1)
               for g in sorted(act_merged[:2197291]['group_1'].unique())]
allCompDays = pd.concat(allCompDays, ignore_index=True)
allCompDays.columns = ['group_1', 'date_act']

CPU times: user 26.5 s, sys: 745 ms, total: 27.3 s
Wall time: 27.4 s


In [7]:
allCompDays.head()

,group_1,date_act
0,group 1,2022-07-17
1,group 1,2022-07-18
2,group 1,2022-07-19
3,group 1,2022-07-20
4,group 1,2022-07-21


In [8]:
meanByCompDays = pd.DataFrame(act_merged.groupby(['group_1', 'date_x'])['outcome'].mean())
meanByCompDays.reset_index(inplace=True)
meanByCompDays.head()

,group_1,date_x,outcome
0,group 1,2022-11-27,0.0
1,group 10,2022-08-19,0.0
2,group 10,2022-08-20,0.0
3,group 10,2022-08-24,0.0
4,group 10,2022-08-25,0.0


In [9]:
allCompDays = allCompDays.merge(meanByCompDays, left_on=['group_1', 'date_act'], 
                                right_on=['group_1', 'date_x'], how='left').drop('date_x', axis=1)
allCompDays.head()

,group_1,date_act,outcome
0,group 1,2022-07-17,NaN
1,group 1,2022-07-18,NaN
2,group 1,2022-07-19,NaN
3,group 1,2022-07-20,NaN
4,group 1,2022-07-21,NaN


In [10]:
# function to interpolate unknown values
def interpolate(x):
    # Find all non-NA indices, combine them with outside borders
    borders = np.array([0] + list(np.where(~pd.isnull(x))[0]) + [len(x)])
    # establish forward and backward - looking indexes
    forward_border = borders[1:]
    backward_border = borders[:-1]
    
    # prepare vectors for filling
    forward_border_x = x.iloc[borders[1:-1]].values
    forward_border_x = np.append(forward_border_x, [abs(forward_border_x[-1] - 0.1)])
    backward_border_x = x.iloc[borders[1:-1]].values
    backward_border_x = np.append([abs(forward_border_x[0] - 0.1)], backward_border_x)
    
    # generate fill vectors
    forward_x_fill = np.repeat(forward_border_x, forward_border-backward_border)
    backward_x_fill = np.repeat(backward_border_x, forward_border-backward_border)
    
    #linear interpolation
    vec = (forward_x_fill + backward_x_fill)/2.
  
    x[pd.isnull(x)] = vec[pd.isnull(x)]
    return x

In [11]:
%%time
out_interp = allCompDays.groupby('group_1')['outcome'].apply(interpolate)
allCompDays['outcome'] = out_interp
del out_interp

CPU times: user 2min 15s, sys: 1.63 s, total: 2min 16s
Wall time: 2min 17s


In [12]:
allCompDays.head()

,group_1,date_act,outcome
0,group 1,2022-07-17,0.05
1,group 1,2022-07-18,0.05
2,group 1,2022-07-19,0.05
3,group 1,2022-07-20,0.05
4,group 1,2022-07-21,0.05


In [13]:
%%time
act_merged = act_merged.merge(allCompDays, left_on=['group_1', 'date_x'], 
                              right_on=['group_1', 'date_act'], how='left')

CPU times: user 4.13 s, sys: 2.23 s, total: 6.35 s
Wall time: 7.05 s


In [14]:
act_merged.head()

,activity_id,people_id,outcome_x,date_x,char_1,group_1,char_2,date_y,char_3,char_4,...,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38,date_act,outcome_y
0,act2_1734928,ppl_100,0.0,2023-08-26,type 2,group 17304,type 2,2021-06-29,type 5,type 5,...,True,False,False,True,True,True,False,36,2023-08-26,0.0
1,act2_2434093,ppl_100,0.0,2022-09-27,type 2,group 17304,type 2,2021-06-29,type 5,type 5,...,True,False,False,True,True,True,False,36,2022-09-27,0.0
2,act2_3404049,ppl_100,0.0,2022-09-27,type 2,group 17304,type 2,2021-06-29,type 5,type 5,...,True,False,False,True,True,True,False,36,2022-09-27,0.0
3,act2_3651215,ppl_100,0.0,2023-08-04,type 2,group 17304,type 2,2021-06-29,type 5,type 5,...,True,False,False,True,True,True,False,36,2023-08-04,0.0
4,act2_4109017,ppl_100,0.0,2023-08-26,type 2,group 17304,type 2,2021-06-29,type 5,type 5,...,True,False,False,True,True,True,False,36,2023-08-26,0.0


In [17]:
act_merged[pd.isnull(act_merged['outcome_y'])]

,activity_id,people_id,outcome_x,date_x,char_1,group_1,char_2,date_y,char_3,char_4,...,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38,date_act,outcome_y
2197513,act2_2277070,ppl_100179,NaN,2022-09-02,type 2,group 23919,type 3,2021-07-22,type 9,type 6,...,False,False,False,False,False,False,False,0,NaT,NaN
2197514,act2_3152044,ppl_100179,NaN,2022-09-02,type 2,group 23919,type 3,2021-07-22,type 9,type 6,...,False,False,False,False,False,False,False,0,NaT,NaN
2197777,act2_1008806,ppl_100322,NaN,2023-01-20,type 2,group 21374,type 3,2021-01-28,type 15,type 6,...,False,False,False,False,False,False,False,70,NaT,NaN
2197778,act2_2022187,ppl_100322,NaN,2023-01-20,type 2,group 21374,type 3,2021-01-28,type 15,type 6,...,False,False,False,False,False,False,False,70,NaT,NaN
2197779,act2_2182495,ppl_100322,NaN,2023-02-18,type 2,group 21374,type 3,2021-01-28,type 15,type 6,...,False,False,False,False,False,False,False,70,NaT,NaN
2197780,act2_2868576,ppl_100322,NaN,2023-02-15,type 2,group 21374,type 3,2021-01-28,type 15,type 6,...,False,False,False,False,False,False,False,70,NaT,NaN
2197781,act2_3947881,ppl_100322,NaN,2022-09-27,type 2,group 21374,type 3,2021-01-28,type 15,type 6,...,False,False,False,False,False,False,False,70,NaT,NaN
2197782,act2_4071983,ppl_100322,NaN,2022-10-13,type 2,group 21374,type 3,2021-01-28,type 15,type 6,...,False,False,False,False,False,False,False,70,NaT,NaN
2197783,act2_4556794,ppl_100322,NaN,2023-02-16,type 2,group 21374,type 3,2021-01-28,type 15,type 6,...,False,False,False,False,False,False,False,70,NaT,NaN
2197784,act2_471550,ppl_100322,NaN,2023-01-11,type 2,group 21374,type 3,2021-01-28,type 15,type 6,...,False,False,False,False,False,False,False,70,NaT,NaN


In [27]:
submit = act_merged[2197291:][['activity_id', 'outcome_y']]
submit['outcome_y'][pd.isnull(submit['outcome_y'])] = submit['outcome_y'][~pd.isnull(submit['outcome_y'])].mean()
submit.columns = ['activity_id', 'outcome']
submit.to_csv('Submission_leak.csv', index=False)

The LB score is ~0.987.